# Semantic Test Case Clustering Demo

This notebook demonstrates the semantic clustering and visualization of test cases using ELMo, BERT, and T5 embeddings.

## Overview

1. Load and preprocess test case data
2. Generate semantic embeddings using different models
3. Apply clustering algorithms
4. Visualize results and evaluate performance

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import project modules
import sys
sys.path.append('../')

from src.data.dataloader import load_demo_data
from src.models.model_factory import ModelFactory
from src.clustering.clustering_engine import ClusteringEngine
from src.clustering.similarity_engine import SimilarityEngine
from src.clustering.dimensionality_reduction import DimensionalityReducer
from src.evaluation.metrics import EvaluationSuite
from src.visualization.interactive_plots import create_cluster_plot
from src.visualization.static_plots import plot_cluster_scatter

print('✅ All modules imported successfully')

## Step 1: Load Test Case Data

In [ ]:
# Load demo data
df = load_demo_data(max_samples=500)

print(f'Loaded {len(df)} test cases')
print(f'Columns: {df.columns.tolist()}')

# Display sample data
df.head()

## Step 2: Generate Embeddings

In [ ]:
# Prepare texts for embedding
texts = df['test_description'].tolist()
text_ids = df['test_case_id'].tolist()

# Test different models
models = ['bert', 'elmo', 't5']
embeddings_dict = {}

for model_name in models:
    print(f'--- Generating {model_name.upper()} embeddings ---')
    
    try:
        # Create model
        config = {'name': model_name, 'embedding_dim': 512}
        embedder = ModelFactory.create_embedder(model_name, config)
        embedder.initialize()
        
        # Generate embeddings
        embeddings = embedder.embed_texts(texts)
        embeddings_dict[model_name] = embeddings
        
        print(f'✅ {model_name.upper()}: {embeddings.shape}')
        
    except Exception as e:
        print(f'❌ Error with {model_name}: {e}')
        # Use random embeddings for demo
        embeddings_dict[model_name] = np.random.randn(len(texts), 512)

## Step 3: Apply Clustering

In [ ]:
# Initialize clustering engine
clustering_engine = ClusteringEngine()
evaluator = EvaluationSuite()

# Test clustering on BERT embeddings
embeddings = embeddings_dict['bert']

# Apply different clustering methods
clustering_results = {}
methods = ['kmeans', 'hierarchical', 'dbscan']

for method in methods:
    print(f'--- {method.upper()} Clustering ---')
    
    if method == 'kmeans':
        labels = clustering_engine.apply_kmeans(embeddings, n_clusters=8)
    elif method == 'hierarchical':
        labels = clustering_engine.apply_hierarchical(embeddings, n_clusters=8)
    else:  # dbscan
        labels = clustering_engine.apply_dbscan(embeddings, eps=0.5)
    
    # Evaluate clustering
    results = evaluator.evaluate_clustering(embeddings, labels)
    clustering_results[method] = {'labels': labels, 'results': results}
    
    print(f'Clusters: {results["n_clusters"]}')
    print(f'Silhouette Score: {results["metrics"].get("silhouette_score", 0):.3f}')
    print(f'Davies-Bouldin: {results["metrics"].get("davies_bouldin_score", 0):.3f}')

## Step 4: Dimensionality Reduction and Visualization

In [ ]:
# Apply t-SNE for visualization
reducer = DimensionalityReducer()
coords_2d = reducer.apply_tsne(embeddings, n_components=2)

print(f'Reduced to 2D coordinates: {coords_2d.shape}')

# Visualize K-means results
kmeans_labels = clustering_results['kmeans']['labels']

# Static plot
plot_cluster_scatter(coords_2d, kmeans_labels, 
                    title='K-means Clustering Results (BERT + t-SNE)',
                    figsize=(12, 8))

In [ ]:
# Interactive plot (if Plotly is available)
try:
    fig = create_cluster_plot(coords_2d, kmeans_labels, texts, 
                             title='Interactive Clustering Visualization')
    if fig:
        fig.show()
    else:
        print('Plotly not available')
except Exception as e:
    print(f'Interactive plot error: {e}')

## Step 5: Semantic Search Demo

In [ ]:
# Initialize similarity engine
similarity_engine = SimilarityEngine()
similarity_engine.load_embeddings(embeddings, text_ids, texts)

# Test semantic search
search_queries = [
    'user login authentication',
    'payment processing validation',
    'database transaction error'
]

for query in search_queries:
    print(f'🔍 Search: "{query}"')
    print('-' * 50)
    
    # Create embedder for query
    config = {'name': 'bert', 'embedding_dim': 512}
    embedder = ModelFactory.create_embedder('bert', config)
    embedder.initialize()
    
    # Search for similar test cases
    results = similarity_engine.search_by_text(query, embedder, top_k=5)
    
    for i, (idx, text_id, score) in enumerate(results, 1):
        print(f'{i}. {text_id} (similarity: {score:.3f})')
        print(f'   {text_id.replace("_", " ").title()}')

## Step 6: Model Comparison

In [ ]:
# Compare different embedding models
model_comparison = {}

for model_name, model_embeddings in embeddings_dict.items():
    print(f'--- Evaluating {model_name.upper()} ---')
    
    # Apply K-means clustering
    clustering_engine = ClusteringEngine()
    labels = clustering_engine.apply_kmeans(model_embeddings, n_clusters=8)
    
    # Evaluate
    results = evaluator.evaluate_clustering(model_embeddings, labels)
    model_comparison[model_name] = results
    
    print(f'Clusters: {results["n_clusters"]}')
    print(f'Silhouette: {results["metrics"].get("silhouette_score", 0):.3f}')

# Create comparison DataFrame
comparison_data = []
for model, results in model_comparison.items():
    metrics = results['metrics']
    comparison_data.append({
        'Model': model.upper(),
        'Silhouette Score': metrics.get('silhouette_score', 0),
        'Davies-Bouldin': metrics.get('davies_bouldin_score', 0),
        'Calinski-Harabasz': metrics.get('calinski_harabasz_score', 0),
        'N Clusters': results['n_clusters']
    })

comparison_df = pd.DataFrame(comparison_data)
print('📊 Model Comparison Results:')
print(comparison_df.round(3))

In [ ]:
# Plot model comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Silhouette scores
axes[0].bar(comparison_df['Model'], comparison_df['Silhouette Score'], 
           color=['skyblue', 'lightcoral', 'lightgreen'])
axes[0].set_title('Silhouette Score Comparison')
axes[0].set_ylabel('Score')

# Davies-Bouldin scores
axes[1].bar(comparison_df['Model'], comparison_df['Davies-Bouldin'],
           color=['skyblue', 'lightcoral', 'lightgreen'])
axes[1].set_title('Davies-Bouldin Index Comparison')
axes[1].set_ylabel('Index (lower is better)')

plt.tight_layout()
plt.show()

## Conclusion

This demo showed:

1. **Data Loading**: Synthetic test case generation for development
2. **Multi-Model Embeddings**: ELMo, BERT, and T5 implementations
3. **Clustering Analysis**: K-means, hierarchical, and DBSCAN methods
4. **Visualization**: 2D t-SNE projections and interactive plots
5. **Semantic Search**: Context-aware test case retrieval
6. **Performance Evaluation**: Comprehensive metrics comparison

The system demonstrates significant potential for improving software testing workflows through semantic understanding of test cases.